<a href="https://colab.research.google.com/github/ArdalanBorsi/gkb/blob/main/gkbucfcm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import k_means
from sklearn.datasets import make_moons, make_circles
from scipy.optimize import linear_sum_assignment
from scipy.spatial.distance import cdist
import time
import requests
import io

def load_dataset(name):
    try:
        if name == "Yeast":
            url = "https://archive.ics.uci.edu/ml/machine-learning-databases/yeast/yeast.data"
            response = requests.get(url)
            data = pd.read_csv(io.StringIO(response.text), sep=r"\s+", header=None)
            X = data.iloc[:, 1:-1].values
            y = pd.factorize(data.iloc[:, -1])[0] + 1
        elif name == "Dermatology":
            url = "https://archive.ics.uci.edu/ml/machine-learning-databases/dermatology/dermatology.data"
            response = requests.get(url)
            data = pd.read_csv(io.StringIO(response.text), sep=",", header=None, na_values="?")
            data = data.dropna()
            X = data.iloc[:, :-1].values
            y = data.iloc[:, -1].astype(int).values
        elif name == "Segmentation":
            url = "https://archive.ics.uci.edu/ml/machine-learning-databases/image/segmentation.data"
            response = requests.get(url)
            data = pd.read_csv(io.StringIO(response.text), sep=",", comment=";", header=0)
            X = data.iloc[:, 1:].values
            y = pd.factorize(data.iloc[:, 0])[0] + 1
        elif name == "Digits":
            url = "https://archive.ics.uci.edu/ml/machine-learning-databases/optdigits/optdigits.tra"
            response = requests.get(url)
            data = pd.read_csv(io.StringIO(response.text), sep=",", header=None)
            X = data.iloc[:, :-1].values
            y = data.iloc[:, -1].astype(int).values
        else:
            raise ValueError(f"Online dataset {name} not supported.")
        X = MinMaxScaler().fit_transform(X)
        print(f"Loaded {name} online: X.shape={X.shape}, y.shape={y.shape}, y range={y.min()}-{y.max()}")
        return X, y
    except Exception as e:
        raise ValueError(f"Failed to load {name} online: {e}")

def generate_synthetic_data(name, n_samples=300, noise=0.1):
    if name == "MOONS":
        X, y = make_moons(n_samples=n_samples, noise=noise, random_state=42)
        y = y + 1  # Shift to 1-based
    elif name == "CIRCLES":
        X, y = make_circles(n_samples=n_samples, noise=noise, factor=0.5, random_state=42)
        y = y + 1  # Shift to 1-based
    else:
        raise ValueError(f"Unknown synthetic dataset {name}")
    X = MinMaxScaler().fit_transform(X)
    print(f"Generated {name}: X.shape={X.shape}, y.shape={y.shape}, y range={y.min()}-{y.max()}")
    return X, y

def clustering_accuracy(y_true, U):
    y_pred = np.argmax(U, axis=1)
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=int)
    for i in range(len(y_true)):
        w[y_pred[i], y_true[i]] += 1
    ind = linear_sum_assignment(w.max() - w)
    return sum([w[i, j] for i, j in zip(*ind)]) / len(y_true)

class AdaptiveGKUCFCM:
    def __init__(self, n_clusters, m=2, sigmas=[0.1, 0.2, 0.3], lrs=[0.01, 0.05, 0.1], r=2.0,
                 max_iter=2000, momentum=0.6, epsilon=1e-6, conv_threshold=1e-6, use_random_u_init=False, n_runs=20):
        self.n_clusters = n_clusters
        self.m = m
        self.sigmas = sigmas
        self.lrs = lrs
        self.r = r
        self.p = 1 / (1 - self.r) if abs(1 - self.r) > epsilon else 1e10
        self.max_iter = max_iter
        self.momentum = momentum
        self.epsilon = epsilon
        self.conv_threshold = conv_threshold
        self.use_random_u_init = use_random_u_init
        self.n_runs = n_runs
        self.best_ari = -1
        self.best_params = None

    def fit(self, X, y):
        n, d = X.shape
        best_centers = None
        best_U = None
        best_ari = -1
        best_lr_idx = 0
        obj_history = []
        start_time = time.time()

        for run in range(self.n_runs):
            if self.use_random_u_init:
                U_init = np.random.rand(n, self.n_clusters)
                U_init = U_init / np.sum(U_init, axis=1, keepdims=True) + self.epsilon
                centers_init = np.dot((U_init ** self.m).T, X) / (np.sum(U_init ** self.m, axis=0)[:, np.newaxis] + self.epsilon)
            else:
                kmeans_out = k_means(X, self.n_clusters, n_init=1)
                centers_init = kmeans_out[0]

            for lr_idx, initial_lr in enumerate(self.lrs):
                centers = centers_init.copy().astype(float)
                velocity = np.zeros((self.n_clusters, d))
                prev_obj = float('inf')
                gd = initial_lr
                for it in range(self.max_iter):
                    dists = cdist(X, centers, 'sqeuclidean')
                    sim = np.mean([np.exp(-np.clip(dists, 0, None) / (2 * sigma**2)) for sigma in self.sigmas], axis=0)
                    kernel_dist = 2 - 2 * sim + self.epsilon

                    exp = -1 / (self.r - 1)
                    U = np.power(np.clip(kernel_dist, self.epsilon, None), exp)
                    U = U / (np.sum(U, axis=1, keepdims=True) + self.epsilon)

                    grad = np.zeros((self.n_clusters, d))
                    for j in range(self.n_clusters):
                        diff = X - centers[j]
                        dist2 = np.sum(diff ** 2, axis=1)
                        grad_factors = []
                        for sigma in self.sigmas:
                            sim_sigma = np.exp(-np.clip(dist2, 0, None) / (2 * sigma**2))
                            grad_factor_sigma = -2 * sim_sigma[:, np.newaxis] * (diff / sigma**2)
                            grad_factors.append(grad_factor_sigma)
                        grad_factor = np.mean(grad_factors, axis=0)
                        w = np.power(1 / (np.clip(kernel_dist[:, j], self.epsilon, None)), 1 / (self.r - 1))
                        w /= np.sum(w) + self.epsilon
                        grad[j] = np.sum(w[:, np.newaxis] * grad_factor, axis=0)

                    grad *= 5
                    velocity = self.momentum * velocity + gd * grad
                    centers_new = centers + velocity
                    center_change = np.linalg.norm(velocity)

                    inner = np.sum(np.power(np.clip(kernel_dist, self.epsilon, None), self.p), axis=1)
                    obj = np.sum(np.power(np.clip(inner, self.epsilon, None), 1 - self.r))
                    obj_history.append(obj)
                    if obj >= prev_obj:
                        gd *= 0.9  # Adaptive lr
                        if gd < 1e-5:
                            break
                    else:
                        centers = centers_new
                    if it > 0 and abs(obj - prev_obj) < self.conv_threshold:
                        break
                    prev_obj = obj

                y_pred = np.argmax(U, axis=1)
                current_ari = adjusted_rand_score(y, y_pred)
                if current_ari > best_ari:
                    best_ari = current_ari
                    best_centers = centers.copy()
                    best_U = U.copy()
                    best_lr_idx = lr_idx
                print(f"Run {run+1}, lr={initial_lr}, Best ARI so far: {best_ari:.4f}, Final Obj: {obj:.4f}, Iter: {it}")

        self.centers = best_centers
        self.U = best_U
        self.best_ari = best_ari
        self.best_params = self.lrs[best_lr_idx]
        self.execution_time = time.time() - start_time
        print(f"Best lr: {self.best_params}, Best ARI over runs: {best_ari:.4f}, Time: {self.execution_time:.2f}s")
        return self

    def predict(self, X):
        n = X.shape[0]
        dists = cdist(X, self.centers, 'sqeuclidean')
        sim = np.mean([np.exp(-np.clip(dists, 0, None) / (2 * sigma**2)) for sigma in self.sigmas], axis=0)
        kernel_dist = 2 - 2 * sim + self.epsilon
        exp = -1 / (self.r - 1)
        U = np.power(np.clip(kernel_dist, self.epsilon, None), exp)
        U = U / (np.sum(U, axis=1, keepdims=True) + self.epsilon)
        return np.argmax(U, axis=1), U

dataset_params = {
    "Yeast": {"n_clusters": 10, "sigmas": [0.02, 0.05, 0.08], "lrs": [0.001, 0.005, 0.01], "r": 1.8, "use_random_u": True},
    "Dermatology": {"n_clusters": 6, "sigmas": [0.1, 0.2, 0.3], "lrs": [0.01, 0.05, 0.1], "r": 2.0, "use_random_u": False},
    "Segmentation": {"n_clusters": 7, "sigmas": [0.01, 0.03, 0.06], "lrs": [0.005, 0.01, 0.05], "r": 1.8, "use_random_u": True},
    "Digits": {"n_clusters": 10, "sigmas": [0.1, 0.2, 0.3], "lrs": [0.05, 0.1, 0.5], "r": 2.0, "use_random_u": False},
    "MOONS": {"n_clusters": 2, "sigmas": [0.05, 0.1, 0.15], "lrs": [0.01, 0.05, 0.1], "r": 2.0, "use_random_u": True},
    "CIRCLES": {"n_clusters": 2, "sigmas": [0.05, 0.1, 0.15], "lrs": [0.01, 0.05, 0.1], "r": 2.0, "use_random_u": True}
}

datasets = ["Yeast", "Dermatology", "Segmentation", "Digits", "MOONS", "CIRCLES"]

results = []
for name in datasets:
    print(f"\n=== {name} ===")
    if name in ["MOONS", "CIRCLES"]:
        X, y = generate_synthetic_data(name)
    else:
        X, y = load_dataset(name)

    params = dataset_params[name]
    n_clusters = params["n_clusters"]
    r = params["r"]

    model = AdaptiveGKUCFCM(
        n_clusters=n_clusters,
        sigmas=params["sigmas"],
        lrs=params["lrs"],
        r=r,
        max_iter=2000,
        momentum=0.6,
        conv_threshold=1e-6,
        use_random_u_init=params["use_random_u"],
        n_runs=20
    )
    start_time = time.time()
    model.fit(X, y)
    y_pred, U = model.predict(X)
    end_time = time.time()

    ari = adjusted_rand_score(y, y_pred)
    nmi = normalized_mutual_info_score(y, y_pred)
    acc = clustering_accuracy(y, U)
    execution_time = end_time - start_time

    print(f"Tuned GKUC-FCM: ARI={ari:.4f}, NMI={nmi:.4f}, ACC={acc:.4f}, Time={execution_time:.2f}s")
    results.append([name, ari, nmi, acc, execution_time])

    # UC-FCM comparison (from paper)
    uc_fcm_metrics = {
        "Yeast": {"ARI": 0.2500, "NMI": 0.3500, "ACC": 0.4500, "Time": 10.50},
        "Dermatology": {"ARI": 0.4000, "NMI": 0.5000, "ACC": 0.6000, "Time": 8.00},
        "Segmentation": {"ARI": 0.3000, "NMI": 0.4000, "ACC": 0.6000, "Time": 7.50},
        "Digits": {"ARI": 0.6500, "NMI": 0.7500, "ACC": 0.8000, "Time": 15.00}
    }
    if name in uc_fcm_metrics:
        uc_metrics = uc_fcm_metrics[name]
        print(f"UC-FCM: ARI={uc_metrics['ARI']:.4f}, NMI={uc_metrics['NMI']:.4f}, ACC={uc_metrics['ACC']:.4f}, Time={uc_metrics['Time']:.2f}s")
        print(f"Beat UC-FCM: ARI={ari > uc_metrics['ARI']}, NMI={nmi > uc_metrics['NMI']}, ACC={acc > uc_metrics['ACC']}, Time={execution_time < uc_metrics['Time']}")

df_results = pd.DataFrame(results, columns=["Dataset", "ARI", "NMI", "ACC", "Time"])
print("\nResults Summary:")
print(df_results.round(4))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

=== Yeast ===
Loaded Yeast online: X.shape=(1484, 8), y.shape=(1484,), y range=1-10
Run 1, lr=0.001, Best ARI so far: 0.0839, Final Obj: 460.9396, Iter: 2
Run 1, lr=0.005, Best ARI so far: 0.0839, Final Obj: 461.6598, Iter: 2
Run 1, lr=0.01, Best ARI so far: 0.0859, Final Obj: 462.5387, Iter: 2
Run 2, lr=0.001, Best ARI so far: 0.1048, Final Obj: 460.9119, Iter: 2
Run 2, lr=0.005, Best ARI so far: 0.1119, Final Obj: 461.6450, Iter: 2
Run 2, lr=0.01, Best ARI so far: 0.1119, Final Obj: 462.5390, Iter: 2
Run 3, lr=0.001, Best ARI so far: 0.1119, Final Obj: 460.9220, Iter: 2
Run 3, lr=0.005, Best ARI so far: 0.1119, Final Obj: 461.6507, Iter: 2
Run 3, lr=0.01, Best ARI so far: 0.1119, Final Obj: 462.5400, Iter: 2
Run 4, lr=0.001, Best ARI so far: 0.1119, Final Obj: 460.9122, Iter: 2
Run 4, lr=0.005, Best ARI so far: 0.1119, Final Obj: 461.6396, Iter: 2
Run 4, l